In [2]:
import os
import tensorflow as tf

In [8]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    


    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity= 100000,
                                         min_after_dequeue= 50000 )
    return images, labels

In [6]:
file_name = "index_HanDB_Train.tfrecord"
tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./"+file_name))
#f=open("./test_record.tfrecord",'rb')

In [4]:
tf_record_reader=tf.TFRecordReader()
_, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

In [5]:
tf_record_features = tf.parse_single_example(
    tf_record_serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'width': tf.FixedLenFeature([], tf.int64),
        'height': tf.FixedLenFeature([], tf.int64),
        'image': tf.FixedLenFeature([], tf.string),
    })

In [9]:
#tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)

#tf_record_image = tf.reshape(tf_record_image, [64,64,1])

#tf_record_label = tf.cast(tf_record_features['label'],tf.string)

images, labels = read_and_decode(tf_record_filename_queue,10)

init=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

record_iterator = tf.python_io.tf_record_iterator(path=file_name)

c=0
for record in record_iterator:
    c+=1
print(c)

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(3):
        img, lab = sess.run([images,labels])
        print(lab)
    coord.request_stop()
    coord.join(threads)



665819
[138  14  45 117  38  69  51  81  87  64]
[132   7  18  57  59  66 112  64 118  42]
[ 50 118  38   4 112 119  91  89  13  69]


In [3]:
def decode(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [4]:
file_name = "Shuffled_HanDB_Train.tfrecord"
train_dataset = tf.data.TFRecordDataset("./"+file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=30000)
train_dataset = train_dataset.batch(100)

In [5]:
train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+file_name))
print(train_data_length)

665890


In [7]:
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
next_image, next_label = iterator.get_next()

print(next_image)
train_init_op = iterator.make_initializer(train_dataset)

total_batch=int(train_data_length/100)

with tf.Session() as sess:
    sess.run(train_init_op)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(2):
        sample_img, sample_lab = sess.run([next_image, next_label])
        print(sample_img)
    coord.request_stop()
    coord.join(threads)

Tensor("IteratorGetNext_1:0", shape=(?, 64, 64, 1), dtype=float32)
[[[[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  ...

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]]


 [[[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  ...

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]]


 [[[255.]
   [255.]
   [255.]
   ...
   [255.]
   [255.]
   [255.]]

  [[255.]
   [255.